# Generate Training Data for Resume Parser Fine-Tuning

This notebook generates training data pairs (resume text → structured JSON) for supervised fine-tuning of the Qwen3 resume parser model.

**Purpose:**
- Processes raw resumes from `combined_resumes.json`
- Extracts structured information using LLM to create (input, output) pairs
- Generates `extracted_resumes.json` with resume text and structured JSON pairs
- Creates the training dataset used for fine-tuning the Qwen3-0.6B model

**Workflow:**
1. Load raw resumes from `combined_resumes.json`
2. For each resume, extract structured JSON using LLM
3. Save resume text + extracted JSON pairs to `extracted_resumes.json`
4. This data is then converted to Qwen3 chat template format for training

**Features:**
- Processes resumes one by one with progress tracking
- Saves progress after each resume (can resume if interrupted)
- Configurable delay between API calls to handle rate limits
- Validates JSON output and tracks errors

**Note:** This notebook uses placeholders for the LLM API configuration. Replace `YOUR_PROVIDER`, `YOUR_MODEL_NAME`, and the LLM client initialization with your actual API setup.


## Setup and Imports


In [ ]:
import json
import time
from pathlib import Path
from datetime import datetime
# TODO: Import your LLM client/API wrapper here
# Example: from your_llm_client import YourLLMClient

print("✓ Imports successful")


## Configuration


In [ ]:
# File paths
INPUT_FILE = "combined_resumes.json"
OUTPUT_FILE = "extracted_resumes.json"
PROGRESS_FILE = "processing_progress.json"

# API configuration
WAIT_TIME_SECONDS = 2
PROGRESS_PRINT_INTERVAL = 20  # Print detailed progress every N resumes

# TODO: Configure your LLM API settings
# Replace these placeholders with your actual API configuration
PROVIDER = "YOUR_PROVIDER"  # e.g., "OpenAI", "Anthropic", etc.
MODEL = "YOUR_MODEL_NAME"   # e.g., "gpt-4", "claude-3", etc.
TEMPERATURE = 0  # Use 0 for deterministic JSON extraction

print(f"Input file: {INPUT_FILE}")
print(f"Output file: {OUTPUT_FILE}")
print(f"Wait time between requests: {WAIT_TIME_SECONDS} seconds")
print(f"Progress update interval: every {PROGRESS_PRINT_INTERVAL} resumes")
print(f"\n⚠️  Note: Update PROVIDER and MODEL with your actual API configuration")


## Prompt Templates


In [ ]:
SYSTEM_PROMPT = """
You are an expert resume information extraction system.

Your task is to extract a structured professional profile from a resume.

STRICT RULES:
- Use ONLY information explicitly present in the resume
- You MAY summarize or rephrase information that is explicitly stated in the resume, but you MUST NOT add new facts or inferred details.
- Do NOT infer or guess missing information
- Do NOT fabricate company names, skills, or achievements
- If a field is not present or cannot be confidently determined, return null
- Normalize job titles and skills where appropriate (e.g., "SDE" → "Software Engineer")

Return ONLY valid JSON.
Do NOT include explanations, comments, or markdown.
"""

def create_user_prompt(resume_text):
    return f"""
Extract a structured professional profile from the resume below.

Schema:
{{
  "current_title": string,
  "previous_titles": string[],
  "current_company": string,
  "previous_companies": string[],
  "years_experience": number,
  "seniority": "junior" | "mid" | "senior" | "lead" | "principal",
  "primary_domain": string,
  "industries": string[],
  "core_skills": string[],
  "secondary_skills": string[],
  "tools": string[],
  "leadership_experience": boolean,
  "key_achievements": string[],
  "location": string,
  "summary": string
}}

Guidelines:
- years_experience should be a number (e.g., 7.5)
- seniority should reflect the highest demonstrated role
- leadership_experience = true only if the resume mentions leading teams, mentoring, managing, or ownership
- key_achievements should be impact-oriented (metrics, outcomes, scale)
- summary must be a concise professional summary (MAX 300 characters)

LIMITING RULES:
- For any array field, return AT MOST the 3 most recent or most relevant values.
- If more than 3 values exist:
  - Job titles and companies: select the 3 most recent based on dates in the resume.
  - Skills, tools, industries: select the 3 most prominent based on frequency and emphasis.
  - Achievements: select the 3 highest-impact achievements.
- Preserve ordering from MOST RECENT or MOST IMPORTANT to LEAST.
- If fewer than 3 values exist, return all available.
- If no values exist, return null.

Resume:
<<<BEGIN RESUME>>>
{resume_text}
<<<END RESUME>>>
"""

print("✓ Prompts configured")


## Helper Functions


In [ ]:
def load_progress():
    """Load processing progress from file."""
    if Path(PROGRESS_FILE).exists():
        with open(PROGRESS_FILE, 'r') as f:
            return json.load(f)
    return {"last_processed_index": -1, "total_processed": 0}

def save_progress(index, total):
    """Save processing progress to file."""
    progress = {
        "last_processed_index": index,
        "total_processed": total,
        "last_update": datetime.now().isoformat()
    }
    with open(PROGRESS_FILE, 'w') as f:
        json.dump(progress, f, indent=2)

def load_existing_results():
    """Load existing results from output file."""
    if Path(OUTPUT_FILE).exists():
        with open(OUTPUT_FILE, 'r') as f:
            return json.load(f)
    return []

def save_result(results):
    """Save results to output file."""
    with open(OUTPUT_FILE, 'w') as f:
        json.dump(results, f, indent=2)

def extract_resume_info(resume_text, llm_client):
    """Extract structured information from a resume using LLM."""
    user_prompt = create_user_prompt(resume_text)
    final_prompt = f"""SYSTEM:
{SYSTEM_PROMPT}

USER:
{user_prompt}
"""
    
    try:
        # TODO: Replace this with your actual LLM API call
        # The structure should:
        # 1. Send the prompt to your LLM API
        # 2. Receive the response text
        # 3. Return the extracted text
        
        # Example structure (adapt to your API):
        # response = llm_client.generate(
        #     prompt=final_prompt,
        #     model=MODEL,
        #     temperature=TEMPERATURE
        # )
        # extracted_text = response["text"]  # or response.text, depending on your API
        
        # Placeholder - replace with actual API call
        response = llm_client.generate(
            prompt=final_prompt,
            model=MODEL,
            temperature=TEMPERATURE
        )
        extracted_text = response["text"]  # Adjust based on your API response structure
        
        # Try to parse as JSON to validate
        try:
            extracted_json = json.loads(extracted_text)
            return extracted_json, None
        except json.JSONDecodeError as e:
            return extracted_text, f"JSON parsing error: {str(e)}"
            
    except Exception as e:
        return None, f"API error: {str(e)}"

print("✓ Helper functions defined")


## Load Input Data


In [ ]:
# Load combined resumes
print(f"Loading resumes from {INPUT_FILE}...")
with open(INPUT_FILE, 'r') as f:
    resumes = json.load(f)

total_resumes = len(resumes)
print(f"✓ Loaded {total_resumes} resumes")

# Load progress
progress = load_progress()
start_index = progress["last_processed_index"] + 1

if start_index > 0:
    print(f"\n⚠️  Resuming from index {start_index}")
    print(f"   Already processed: {progress['total_processed']} resumes")
else:
    print(f"\n▶️  Starting fresh processing")

remaining = total_resumes - start_index
print(f"   Remaining to process: {remaining} resumes")


## Process Resumes

**Main Processing Loop** - This cell will process all remaining resumes with rate limiting and progress tracking.


In [ ]:
# TODO: Initialize your LLM client
# Replace this with your actual LLM client initialization
# Example: llm_client = YourLLMClient(api_key="your_key")
llm_client = None  # Placeholder - initialize your LLM client here
print("✓ LLM client initialized\n")

# Load existing results
results = load_existing_results()
print(f"Loaded {len(results)} existing results\n")

# Process resumes
print("="*70)
print("STARTING PROCESSING")
print("="*70)

successful_count = 0
error_count = 0
start_time = time.time()

for i in range(start_index, total_resumes):
    resume_data = resumes[i]
    resume_text = resume_data.get("resume_text", "")
    
    # Determine if we should print progress for this resume
    should_print = ((i - start_index + 1) % PROGRESS_PRINT_INTERVAL == 0) or (i == start_index) or (i == total_resumes - 1)
    
    if should_print:
        print(f"\n[{i+1}/{total_resumes}] Processing resume...")
    
    if not resume_text or resume_text.strip() == "":
        if should_print:
            print("  ⚠️  Empty resume text, skipping")
        results.append({
            "index": i,
            "resume_text": resume_text,
            "extracted_json": None,
            "error": "Empty resume text",
            "processed_at": datetime.now().isoformat()
        })
        error_count += 1
        continue
    
    # Extract information
    extracted_json, error = extract_resume_info(resume_text, llm_client)
    
    # Prepare result
    result = {
        "index": i,
        "resume_text": resume_text,
        "extracted_json": extracted_json,
        "error": error,
        "processed_at": datetime.now().isoformat()
    }
    
    results.append(result)
    
    # Update counters
    if error:
        error_count += 1
        if should_print:
            print(f"  ❌ Error: {error}")
    else:
        successful_count += 1
        if should_print:
            print(f"  ✓ Successfully extracted")
    
    # Save progress after each resume
    save_result(results)
    save_progress(i, len(results))
    
    # Progress summary - only print every N resumes
    if should_print:
        elapsed = time.time() - start_time
        avg_time = elapsed / (i - start_index + 1)
        remaining_count = total_resumes - i - 1
        eta_seconds = avg_time * remaining_count
        eta_minutes = eta_seconds / 60
        
        print(f"  Progress: {len(results)}/{total_resumes} | Success: {successful_count} | Errors: {error_count}")
        print(f"  Avg time: {avg_time:.2f}s | ETA: {eta_minutes:.1f} minutes")
    
    # Wait before next request (except for the last one)
    if i < total_resumes - 1:
        time.sleep(WAIT_TIME_SECONDS)

# Final summary
print("\n" + "="*70)
print("PROCESSING COMPLETE")
print("="*70)
print(f"\nTotal processed: {len(results)}")
print(f"Successful: {successful_count}")
print(f"Errors: {error_count}")
print(f"\nResults saved to: {OUTPUT_FILE}")
print(f"Total time: {(time.time() - start_time)/60:.1f} minutes")


## View Sample Results


In [ ]:
# Load and display first few successful results
with open(OUTPUT_FILE, 'r') as f:
    all_results = json.load(f)

# Find first successful result
successful_results = [r for r in all_results if r['error'] is None]

if successful_results:
    print(f"Showing first successful result (out of {len(successful_results)} total):")
    print("\n" + "="*70)
    
    sample = successful_results[0]
    print(f"Resume Index: {sample['index']}")
    print(f"Processed At: {sample['processed_at']}")
    print("\nExtracted JSON:")
    print(json.dumps(sample['extracted_json'], indent=2))
    
    print("\n" + "="*70)
    print(f"\nResume Text (first 500 chars):")
    print(sample['resume_text'][:500] + "...")
else:
    print("No successful results found yet.")


## Statistics and Analysis


In [ ]:
# Analyze results
with open(OUTPUT_FILE, 'r') as f:
    all_results = json.load(f)

total = len(all_results)
successful = sum(1 for r in all_results if r['error'] is None)
failed = total - successful

print("PROCESSING STATISTICS")
print("="*70)
print(f"Total processed: {total}")
print(f"Successful: {successful} ({successful/total*100:.1f}%)")
print(f"Failed: {failed} ({failed/total*100:.1f}%)")

if failed > 0:
    print("\nError Summary:")
    errors = {}
    for r in all_results:
        if r['error']:
            error_type = r['error'].split(':')[0]
            errors[error_type] = errors.get(error_type, 0) + 1
    
    for error_type, count in sorted(errors.items(), key=lambda x: x[1], reverse=True):
        print(f"  - {error_type}: {count}")
